<a href="https://colab.research.google.com/github/Kussil/CVX_Rice_project/blob/main/02_Cleaned_Data/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
from google.colab import userdata
import pandas as pd

In [2]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/CVX_Rice_project.git

Cloning into 'CVX_Rice_project'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 287 (delta 47), reused 61 (delta 31), pack-reused 200
Receiving objects: 100% (287/287), 105.84 MiB | 7.25 MiB/s, done.
Resolving deltas: 100% (147/147), done.
Updating files: 100% (23/23), done.


## Upload Investment Research Articles into DF

In [3]:
# Import Chevron and Other Majors CSVs
invest_df = pd.read_csv('CVX_Rice_project/01_Raw_Data/Investment_Research_ALLV2_Final.csv')

# Rename drop, and reorder columns
invest_df = invest_df.rename(columns={'Headline': 'Article Headline', 'Text': 'Article Text'})
invest_df = invest_df.drop(['Contributor', 'Date/Time'], axis=1)
invest_df['Source'] = 'Investment Research'
new_order = ['Source', 'Unique_ID', 'Ticker', 'Date', 'Article Headline', 'Article Text', 'URL']
invest_df = invest_df.reindex(columns=new_order)
display(invest_df.shape)
display(invest_df.head())

(4896, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


## Upload Proquest News Articles into DF and Clean Data

In [4]:
# Import Files
proquest_df_cvx = pd.read_csv('CVX_Rice_project/01_Raw_Data/Proquest_Articles_CVX.csv')
proquest_df_xom = pd.read_csv('CVX_Rice_project/01_Raw_Data/ProQuest_Articles_XOM.csv')
proquest_df_others = pd.read_csv('CVX_Rice_project/01_Raw_Data/ProQuest_Articles_FINAL_BP_COP_OXY_SHEL.csv')

# Append dfs
proquest_df = pd.concat([proquest_df_cvx,
                         proquest_df_xom,
                         proquest_df_others
                         ], ignore_index=True)
display(proquest_df_cvx.shape)
display(proquest_df_xom.shape)
display(proquest_df_others.shape)
display(proquest_df.shape)

# Rename drop, and reorder columns
proquest_df = proquest_df.rename(columns={'Title': 'Article Headline', 'Full Article Text': 'Article Text'})
proquest_df['Source'] = 'ProQuest'
proquest_df = proquest_df.reindex(columns=new_order)

display(proquest_df.head())

(261, 6)

(1478, 6)

(1445, 6)

(3184, 6)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,ProQuest,2703147472,CVX,17-Aug-22,Oil Giants Must Face Climate-Liability Suits i...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-ge...
1,ProQuest,2702197510,CVX,15-Aug-22,Warren Buffett's Berkshire Hathaway Keeps Spen...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-sl...
2,ProQuest,2697049245,CVX,2-Aug-22,"Investors Put Forward More Proposals, Dialing ...",Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-sl...
3,ProQuest,2696371267,CVX,30-Jul-22,Soaring Fuel Prices Catapult Oil Giants to Rec...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/senators-a...
4,ProQuest,2695835797,CVX,29-Jul-22,"Exxon, Chevron, Shell Report Record Profits on...",Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/shell-trie...


In [5]:
# Delete rows with missing article text
search_text_1 = 'Failed to load content: Message:'
search_text_2 = 'Relevant content not found within the specified range.'
search_text_3 = 'TVM:UNDEFINED'

# Count the number of rows containing either search text
count_rows = proquest_df[
    proquest_df['Article Text'].str.contains(search_text_1, na=False) |
    proquest_df['Article Text'].str.contains(search_text_2, na=False) |
    proquest_df['Article Text'].str.contains(search_text_3, na=False)
].shape[0]

# Delete the rows containing either search text
proquest_df = proquest_df[
    ~proquest_df['Article Text'].str.contains(search_text_1, na=False) &
    ~proquest_df['Article Text'].str.contains(search_text_2, na=False) &
    ~proquest_df['Article Text'].str.contains(search_text_3, na=False)
]

# Print the number of rows with missing article text and the new shape of the DataFrame
print(f"Number of rows with missing article text: {count_rows}")
print(f"New DataFrame shape: {proquest_df.shape}")
print()

# Confirm data is good by looking for short article headlines
shortest_headline = proquest_df.loc[proquest_df['Article Headline'].str.len().idxmin(), 'Article Headline']
print(f"The shortest article headline is: '{shortest_headline}'")
print()

# Confirm data is good by looking for short article text
shortest_text = proquest_df.loc[proquest_df['Article Text'].str.len().idxmin(), 'Article Text']
print(f"The shortest article text is: '{shortest_text}'")

Number of rows with missing article text: 187
New DataFrame shape: (2997, 7)

The shortest article headline is: 'BP PLC'

The shortest article text is: 'Turn on search term navigationTurn on search term navigation
| Jump to first hitEXXON Mobil has revealed full-year profits of $23bn (£17bn), following a strong fourthquarter of trading amid successful cost cutting measures and share buyback plans. The results will be relief for the fossil fuel giant amid sustained activist pressure.'


In [6]:
# Clean extra text from the beginning of the article
def clean_text(text):
    if text.startswith("Turn on search term"):
            return text[80:].strip()
    return text
proquest_df['Article Text'] = proquest_df['Article Text'].apply(clean_text)

# Confirm data is good by looking for short article headlines
shortest_headline = proquest_df.loc[proquest_df['Article Headline'].str.len().idxmin(), 'Article Headline']
print(f"The shortest article headline is: '{shortest_headline}'")
print()

# Confirm data is good by looking for short article text
shortest_text = proquest_df.loc[proquest_df['Article Text'].str.len().idxmin(), 'Article Text']
print(f"The shortest article text is: '{shortest_text}'")

display(proquest_df.head())

The shortest article headline is: 'BP PLC'

The shortest article text is: 'EXXON Mobil has revealed full-year profits of $23bn (£17bn), following a strong fourthquarter of trading amid successful cost cutting measures and share buyback plans. The results will be relief for the fossil fuel giant amid sustained activist pressure.'


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,ProQuest,2703147472,CVX,17-Aug-22,Oil Giants Must Face Climate-Liability Suits i...,Chevron Corp.\nSpending projections are slidin...,https://www.proquest.com/newspapers/chevron-ge...
1,ProQuest,2702197510,CVX,15-Aug-22,Warren Buffett's Berkshire Hathaway Keeps Spen...,Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
2,ProQuest,2697049245,CVX,2-Aug-22,"Investors Put Forward More Proposals, Dialing ...",Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
3,ProQuest,2696371267,CVX,30-Jul-22,Soaring Fuel Prices Catapult Oil Giants to Rec...,SENATORS aired misgivings on Tuesday over pote...,https://www.proquest.com/newspapers/senators-a...
4,ProQuest,2695835797,CVX,29-Jul-22,"Exxon, Chevron, Shell Report Record Profits on...",LONDON—Royal Dutch Shell PLC said it would sta...,https://www.proquest.com/newspapers/shell-trie...


In [7]:
# Look for duplicate rows
duplicates = proquest_df[proquest_df.duplicated(subset=['Ticker', 'Date', 'Article Headline'], keep=False)]
duplicate_count = duplicates.shape[0]
print(f"Number of duplicate rows: {duplicate_count}")
display(duplicates.head(6))
print()

# Drop duplicates and keep the first occurrence
proquest_df = proquest_df.drop_duplicates(subset=['Ticker', 'Date', 'Article Headline'], keep='first')
display(proquest_df.shape)

Number of duplicate rows: 246


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
69,ProQuest,2774491352,CVX,27-May-21,Chevron corp. investors back climate proposal;...,Russia is still finding a home for much of its...,https://www.proquest.com/newspapers/business-n...
70,ProQuest,2533063981,CVX,27-May-21,Chevron corp. investors back climate proposal;...,Russia is still finding a home for much of its...,https://www.proquest.com/newspapers/chevron-ce...
94,ProQuest,2483968356,CVX,30-Jan-21,Chevron posts fourth-quarter loss on weak refi...,[Financial Analysis and Commentary]\nChevron w...,https://www.proquest.com/newspapers/exchange-h...
95,ProQuest,2483951085,CVX,30-Jan-21,Chevron posts fourth-quarter loss on weak refi...,Chevron Corp. had its most profitable year sin...,https://www.proquest.com/newspapers/exchange-c...
105,ProQuest,2455124806,CVX,28-Oct-20,Chevron to lay off about 25% of noble energy e...,Michael Wirth Chevron Corporation has issued a...,https://www.proquest.com/newspapers/chevron-se...
106,ProQuest,2455119828,CVX,28-Oct-20,Chevron to lay off about 25% of noble energy e...,"Chevron Corp. committed to an ""aspiration"" of ...",https://www.proquest.com/newspapers/chevron-em...


(2840, 7)

In [8]:
# Tail by itself so you can examine the table with the colab table button
display(proquest_df.tail(20))

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
3164,ProQuest,2291578214,COP,"Sep 17, 2019",ConocoPhillips Could be the Best Oil Stock to ...,Investors in general have avoided energy stock...,https://www.proquest.com/newspapers/conocophil...
3165,ProQuest,2290860281,COP,"Sep 16, 2019",Oil Stocks Are Flying After Attack on Saudi Ar...,Oil futures spiked 10% Monday morning followin...,https://www.proquest.com/newspapers/oil-stocks...
3166,ProQuest,2281958248,COP,"Aug 30, 2019",Potential $50b wave of gas M&A deals around th...,Almost $50 billion of oil and gas assets could...,https://www.proquest.com/newspapers/potential-...
3167,ProQuest,2280320507,COP,"Aug 26, 2019",US$30-billion oilsands exodus marches on; Afte...,Capital keeps marching out of Canada's oil ind...,https://www.proquest.com/newspapers/us-30-bill...
3168,ProQuest,2280273929,COP,"Aug 26, 2019",The US$30B exodus; Foreign oil firms are baili...,Capital keeps marching out of Canada's oil ind...,https://www.proquest.com/newspapers/us-30b-exo...
3169,ProQuest,2269386903,COP,"Aug 9, 2019",ConocoPhillips mulls northern Australian gas s...,US major ConocoPhillips is thought to be ponde...,https://www.proquest.com/newspapers/conocophil...
3170,ProQuest,2266372517,COP,"Jul 30, 2019",ConocoPhillips Stock Is Slipping After Missing...,ConocoPhillips\nposted earnings\nthat missed a...,https://www.proquest.com/newspapers/conocophil...
3171,ProQuest,2257230128,COP,"Jul 15, 2019",ConocoPhillips eyes Capital Square for lease,Exploration and production company ConocoPhill...,https://www.proquest.com/newspapers/conocophil...
3172,ProQuest,2256171063,COP,"Jul 12, 2019",Chevron and ConocoPhillips Are the 2 Best Oil ...,Chevron and ConocoPhillips look like the best ...,https://www.proquest.com/newspapers/chevron-co...
3173,ProQuest,2246954300,COP,"Jun 26, 2019","ConocoPhillips Stock Gains, Boeing Flies, 3 Mo...","A Quiet Place. On the surface, the\nstock mark...",https://www.proquest.com/newspapers/conocophil...


## Concatenate both DF Sources and Export to CSV

In [9]:
# Concatenate
text_df = pd.concat([invest_df, proquest_df], ignore_index=True)
display(text_df.shape)
display(text_df.head())

(7736, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


In [10]:
# Get a count of articles by source and ticker
grouped_counts = text_df.groupby(['Source', 'Ticker']).size().reset_index(name='Count')
print(grouped_counts)

                 Source                 Ticker  Count
0   Investment Research                     BP    277
1   Investment Research                    COP    283
2   Investment Research                    CVX    363
3   Investment Research  Concho Resources Inc.    143
4   Investment Research                    DVN    279
5   Investment Research                    EOG    296
6   Investment Research                   EQNR      5
7   Investment Research                    HES    279
8   Investment Research                    MPC    346
9   Investment Research                    MRO    277
10  Investment Research                    OXY    279
11  Investment Research                   PDCE    305
12  Investment Research                    PSX    277
13  Investment Research                    PXD    320
14  Investment Research                   SHEL    281
15  Investment Research                    TTE    255
16  Investment Research                    VLO    293
17  Investment Research     

In [11]:
# Export as CSV
text_df.to_csv('/content/Consolidated_Text_Data.csv', index=False)

# Note: This export needs to be manually uploaded to Github.